In [3]:
import requests
import pandas as pd
import time

import os

from dotenv import load_dotenv
load_dotenv()  # Loads env variable from the root .env file


'792e5f0bdf4f464099e113024212911'

In [2]:
time.sleep(2)

In [4]:
key = os.getenv('WEATHERKEY')

base_url = 'http://api.weatherapi.com/v1'
api = '/history.json'
url = base_url + api

In [6]:
# Test historique sur caluire

q = '45.796609, 4.832035' # coord GPS
dt = '2021-11-25' # Start Date
end_dt = '2021-11-28' # End date

In [7]:
params = {
    'key':key,
    'q':q,
    'dt':dt,
    'end_dt':end_dt,
}

response = requests.get(url,params=params)

In [8]:
response

<Response [200]>

In [9]:
jresp = response.json()

In [10]:
forecasts = jresp.get('forecast').get('forecastday')

for f in forecasts :
    print('---------------------')
    print(f.get('date'))
    print(f.get('day').get('avgtemp_c'))
    print(f.get('day').get('totalprecip_mm'))
    

---------------------
2021-11-25
6.0
0.3
---------------------
2021-11-26
3.1
3.5
---------------------
2021-11-27
3.8
1.3
---------------------
2021-11-28
2.9
4.4


In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from water_pollution.data.preproc import *
from water_pollution.data import stations
from water_pollution.data import stations

from water_pollution.data import visu

from datetime import timedelta
from dateutil.relativedelta import relativedelta

In [43]:
# Builds the small training df (from saone data between 2011-2021)
file_path = '../../raw_data/pc_saone_2011_2021/Analyses.CSV'
rawdf = get_rawdf_from_file(file_path)
da_df = build_saone_base_training_data(rawdf)

In [44]:
stationsdf = stations.get_saone_stations_df()

In [55]:
bystationdfs = {}
station_ids = list(da_df['station_id'].unique())

for st_id in station_ids :
    st_bool = da_df['station_id'] == st_id
    st_df = da_df[st_bool].sort_values('date')
    bystationdfs[st_id] = st_df
    

## Recuperation Precipitation - test sur caluire

In [153]:
!pwd

/home/sve/code/sebvey/water_pollution/notebooks/sebvey


In [155]:
# API Defaults
key = 'renseigner api key' # API sve key

base_url = 'http://api.weatherapi.com/v1'
api = '/history.json'
url = base_url + api

# Save Path
save_path = '/home/sve/code/sebvey/water_pollution/cooked_data/saone/weather/'

In [184]:
# Dictionnaire des stations
stations_dict = stations.get_saone_stations_dict() # pour récupérer les coord
stations_id_list = list(bystationdfs.keys()) # les stations pour lesquelles on a de la données
stations_id_list.sort()

In [185]:
stations_id_list

[6000990,
 6000993,
 6000998,
 6001000,
 6002500,
 6003600,
 6005500,
 6011000,
 6017050,
 6017070,
 6037400,
 6039500,
 6045800,
 6053800,
 6059500,
 6810010]

In [186]:
for st_id in stations_id_list[3:] :
    
    ## POUR UNE STATION
    print('Station_id : ' + str(st_id))

    # GPS coordinates
    coord = stations_dict.get(st_id).get('coord')
    q = str(coord[0]) + ',' + str(coord[1])

    ## DECOUPAGE DES REQUETES PAR MOIS

    st1df = bystationdfs.get(st_id)
    first_date = st1df['date'].min() - relativedelta(months=2)
    last_date = st1df['date'].max()

    dt_starts_range = pd.date_range(start=first_date,end=last_date,freq='MS')
    dt_stops_range = pd.date_range(start=dt_starts_range[0],end=dt_starts_range[-1],freq='M')

    dt_starts_str = [ d.strftime('%Y-%m-%d') for d in dt_starts_range ]
    dt_stops_str = [ d.strftime('%Y-%m-%d') for d in dt_stops_range ]

    # each element is the first and the last day of a month (dt,dt_end)
    month_dts = tuple(zip(dt_starts_str,dt_stops_str))
    
    # Liste des forecast de la station
    forecast_list = [] # ('date','avgtemp_c','totalprecip_mm')
    
    # Pour chaque mois
    for month in month_dts :
        print('-- ' + str(month))
        time.sleep(0.1)

        dt = month[0]
        end_dt = month[1]

        params = {
            'key':key,
            'q':q,
            'dt':dt,
            'end_dt':end_dt,
        }

        response = requests.get(url,params=params)
        
        while not response : # si code != 400 on attend et on recommence
            time.sleep(2)
            response = requests.get(url,params=params)
            
        jresp = response.json()

        forecasts = jresp.get('forecast').get('forecastday')



        for f in forecasts :
            forecast_list.append( ( f.get('date'),
                                    f.get('day').get('avgtemp_c'),
                                    f.get('day').get('totalprecip_mm') ) )
    
    # Formattage et écriture des données pour la station :
    forecast_df = pd.DataFrame(forecast_list)
    forecast_df.columns = ['date','temp','precipitation']
    forecast_df['date'] = pd.to_datetime(forecast_df['date']) 
    forecast_df.set_index('date',inplace=True)

    file_path = save_path + str(st_id) + '.csv'
    forecast_df.to_csv(file_path)
    


Station_id : 6001000
-- ('2011-11-01', '2011-11-30')
-- ('2011-12-01', '2011-12-31')
-- ('2012-01-01', '2012-01-31')
-- ('2012-02-01', '2012-02-29')
-- ('2012-03-01', '2012-03-31')
-- ('2012-04-01', '2012-04-30')
-- ('2012-05-01', '2012-05-31')
-- ('2012-06-01', '2012-06-30')
-- ('2012-07-01', '2012-07-31')
-- ('2012-08-01', '2012-08-31')
-- ('2012-09-01', '2012-09-30')
-- ('2012-10-01', '2012-10-31')
-- ('2012-11-01', '2012-11-30')
-- ('2012-12-01', '2012-12-31')
-- ('2013-01-01', '2013-01-31')
-- ('2013-02-01', '2013-02-28')
-- ('2013-03-01', '2013-03-31')
-- ('2013-04-01', '2013-04-30')
-- ('2013-05-01', '2013-05-31')
-- ('2013-06-01', '2013-06-30')
-- ('2013-07-01', '2013-07-31')
-- ('2013-08-01', '2013-08-31')
-- ('2013-09-01', '2013-09-30')
-- ('2013-10-01', '2013-10-31')
-- ('2013-11-01', '2013-11-30')
-- ('2013-12-01', '2013-12-31')
-- ('2014-01-01', '2014-01-31')
-- ('2014-02-01', '2014-02-28')
-- ('2014-03-01', '2014-03-31')
-- ('2014-04-01', '2014-04-30')
-- ('2014-05-01', '

-- ('2014-03-01', '2014-03-31')
-- ('2014-04-01', '2014-04-30')
-- ('2014-05-01', '2014-05-31')
-- ('2014-06-01', '2014-06-30')
-- ('2014-07-01', '2014-07-31')
-- ('2014-08-01', '2014-08-31')
-- ('2014-09-01', '2014-09-30')
-- ('2014-10-01', '2014-10-31')
-- ('2014-11-01', '2014-11-30')
-- ('2014-12-01', '2014-12-31')
-- ('2015-01-01', '2015-01-31')
-- ('2015-02-01', '2015-02-28')
-- ('2015-03-01', '2015-03-31')
-- ('2015-04-01', '2015-04-30')
-- ('2015-05-01', '2015-05-31')
-- ('2015-06-01', '2015-06-30')
-- ('2015-07-01', '2015-07-31')
-- ('2015-08-01', '2015-08-31')
-- ('2015-09-01', '2015-09-30')
-- ('2015-10-01', '2015-10-31')
-- ('2015-11-01', '2015-11-30')
-- ('2015-12-01', '2015-12-31')
-- ('2016-01-01', '2016-01-31')
-- ('2016-02-01', '2016-02-29')
-- ('2016-03-01', '2016-03-31')
-- ('2016-04-01', '2016-04-30')
-- ('2016-05-01', '2016-05-31')
-- ('2016-06-01', '2016-06-30')
-- ('2016-07-01', '2016-07-31')
-- ('2016-08-01', '2016-08-31')
-- ('2016-09-01', '2016-09-30')
-- ('201

-- ('2015-12-01', '2015-12-31')
-- ('2016-01-01', '2016-01-31')
-- ('2016-02-01', '2016-02-29')
-- ('2016-03-01', '2016-03-31')
-- ('2016-04-01', '2016-04-30')
-- ('2016-05-01', '2016-05-31')
-- ('2016-06-01', '2016-06-30')
-- ('2016-07-01', '2016-07-31')
-- ('2016-08-01', '2016-08-31')
-- ('2016-09-01', '2016-09-30')
-- ('2016-10-01', '2016-10-31')
-- ('2016-11-01', '2016-11-30')
-- ('2016-12-01', '2016-12-31')
-- ('2017-01-01', '2017-01-31')
-- ('2017-02-01', '2017-02-28')
-- ('2017-03-01', '2017-03-31')
-- ('2017-04-01', '2017-04-30')
-- ('2017-05-01', '2017-05-31')
-- ('2017-06-01', '2017-06-30')
-- ('2017-07-01', '2017-07-31')
-- ('2017-08-01', '2017-08-31')
-- ('2017-09-01', '2017-09-30')
-- ('2017-10-01', '2017-10-31')
-- ('2017-11-01', '2017-11-30')
-- ('2017-12-01', '2017-12-31')
-- ('2018-01-01', '2018-01-31')
-- ('2018-02-01', '2018-02-28')
-- ('2018-03-01', '2018-03-31')
-- ('2018-04-01', '2018-04-30')
-- ('2018-05-01', '2018-05-31')
-- ('2018-06-01', '2018-06-30')
-- ('201

-- ('2017-08-01', '2017-08-31')
-- ('2017-09-01', '2017-09-30')
-- ('2017-10-01', '2017-10-31')
-- ('2017-11-01', '2017-11-30')
-- ('2017-12-01', '2017-12-31')
-- ('2018-01-01', '2018-01-31')
-- ('2018-02-01', '2018-02-28')
-- ('2018-03-01', '2018-03-31')
-- ('2018-04-01', '2018-04-30')
-- ('2018-05-01', '2018-05-31')
-- ('2018-06-01', '2018-06-30')
-- ('2018-07-01', '2018-07-31')
-- ('2018-08-01', '2018-08-31')
-- ('2018-09-01', '2018-09-30')
-- ('2018-10-01', '2018-10-31')
-- ('2018-11-01', '2018-11-30')
-- ('2018-12-01', '2018-12-31')
-- ('2019-01-01', '2019-01-31')
-- ('2019-02-01', '2019-02-28')
-- ('2019-03-01', '2019-03-31')
-- ('2019-04-01', '2019-04-30')
-- ('2019-05-01', '2019-05-31')
-- ('2019-06-01', '2019-06-30')
-- ('2019-07-01', '2019-07-31')
-- ('2019-08-01', '2019-08-31')
-- ('2019-09-01', '2019-09-30')
-- ('2019-10-01', '2019-10-31')
-- ('2019-11-01', '2019-11-30')
-- ('2019-12-01', '2019-12-31')
-- ('2020-01-01', '2020-01-31')
-- ('2020-02-01', '2020-02-29')
-- ('202

-- ('2019-05-01', '2019-05-31')
-- ('2019-06-01', '2019-06-30')
-- ('2019-07-01', '2019-07-31')
-- ('2019-08-01', '2019-08-31')
-- ('2019-09-01', '2019-09-30')
-- ('2019-10-01', '2019-10-31')
-- ('2019-11-01', '2019-11-30')
-- ('2019-12-01', '2019-12-31')
-- ('2020-01-01', '2020-01-31')
-- ('2020-02-01', '2020-02-29')
-- ('2020-03-01', '2020-03-31')
-- ('2020-04-01', '2020-04-30')
-- ('2020-05-01', '2020-05-31')
-- ('2020-06-01', '2020-06-30')
-- ('2020-07-01', '2020-07-31')
-- ('2020-08-01', '2020-08-31')
-- ('2020-09-01', '2020-09-30')
-- ('2020-10-01', '2020-10-31')
-- ('2020-11-01', '2020-11-30')
-- ('2020-12-01', '2020-12-31')
-- ('2021-01-01', '2021-01-31')
-- ('2021-02-01', '2021-02-28')
-- ('2021-03-01', '2021-03-31')
-- ('2021-04-01', '2021-04-30')
-- ('2021-05-01', '2021-05-31')
-- ('2021-06-01', '2021-06-30')
-- ('2021-07-01', '2021-07-31')
Station_id : 6045800
-- ('2011-11-01', '2011-11-30')
-- ('2011-12-01', '2011-12-31')
-- ('2012-01-01', '2012-01-31')
-- ('2012-02-01', '

-- ('2021-02-01', '2021-02-28')
-- ('2021-03-01', '2021-03-31')
-- ('2021-04-01', '2021-04-30')
-- ('2021-05-01', '2021-05-31')
-- ('2021-06-01', '2021-06-30')
-- ('2021-07-01', '2021-07-31')
Station_id : 6059500
-- ('2011-10-01', '2011-10-31')
-- ('2011-11-01', '2011-11-30')
-- ('2011-12-01', '2011-12-31')
-- ('2012-01-01', '2012-01-31')
-- ('2012-02-01', '2012-02-29')
-- ('2012-03-01', '2012-03-31')
-- ('2012-04-01', '2012-04-30')
-- ('2012-05-01', '2012-05-31')
-- ('2012-06-01', '2012-06-30')
-- ('2012-07-01', '2012-07-31')
-- ('2012-08-01', '2012-08-31')
-- ('2012-09-01', '2012-09-30')
-- ('2012-10-01', '2012-10-31')
-- ('2012-11-01', '2012-11-30')
-- ('2012-12-01', '2012-12-31')
-- ('2013-01-01', '2013-01-31')
-- ('2013-02-01', '2013-02-28')
-- ('2013-03-01', '2013-03-31')
-- ('2013-04-01', '2013-04-30')
-- ('2013-05-01', '2013-05-31')
-- ('2013-06-01', '2013-06-30')
-- ('2013-07-01', '2013-07-31')
-- ('2013-08-01', '2013-08-31')
-- ('2013-09-01', '2013-09-30')
-- ('2013-10-01', '

In [187]:
response

<Response [200]>